Claro, aquí tienes una guía paso a paso en castellano, con explicaciones, diagramas conceptuales y el código de las imágenes, todo listo para usar en tu archivo tests.ipynb. Incluye explicaciones detalladas y una receta de implementación para realizar los testeos de particionado de datos en Machine Learning.

Similar code found with 1 license type - View matches

# Cómo crear y validar conjuntos de entrenamiento y testeo en Machine Learning

## 1. ¿Por qué separar un conjunto de testeo?

Cuando desarrollamos modelos de Machine Learning, es fundamental reservar una parte de los datos para evaluar el rendimiento real del modelo. Si usamos todos los datos para entrenar, podríamos sobreajustar el modelo y obtener resultados poco realistas.

**Mapa conceptual:**

```mermaid
graph TD
    A[Datos originales] --> B[Separar en entrenamiento y test]
    B --> C[Entrenar modelo con entrenamiento]
    B --> D[Testear modelo con test]
    C --> E[Evaluar rendimiento real]
    D --> E
```

---

## 2. Separación aleatoria simple

Primero, veamos cómo separar los datos de forma aleatoria usando Numpy.

```python
import numpy as np

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]
```

**Explicación paso a paso:**
1. Mezclamos los índices de los datos aleatoriamente.
2. Calculamos el tamaño del conjunto de test.
3. Seleccionamos los primeros índices para test y el resto para entrenamiento.

---

## 3. Problemas de la separación aleatoria

Si ejecutamos varias veces la función anterior, obtendremos diferentes conjuntos de testeo. Esto puede causar inconsistencias en la evaluación.

**Solución:** Usar un identificador único para cada fila y una función de hash para decidir si va a test o entrenamiento.

```python
import hashlib

def test_set_check(identifier, test_ratio, hash):
    return hash(np.int64(identifier)).digest()[-1] < 256 * test_ratio

def split_train_test_by_id(data, test_ratio, id_column, hash=hashlib.md5):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio, hash))
    return data.loc[~in_test_set], data.loc[in_test_set]
```

**Receta de implementación:**
1. Añade una columna de identificador único (puede ser el índice o una combinación de columnas estables).
2. Usa la función anterior para separar los datos de forma consistente.

---

## 4. Estratificación: asegurar proporciones representativas

A veces, es importante que ciertas categorías (por ejemplo, rangos de ingresos) estén representadas proporcionalmente en ambos conjuntos.

**Pasos:**
1. Crear una nueva columna categórica (por ejemplo, `income_cat`).
2. Usar `StratifiedShuffleSplit` de Scikit-Learn para dividir los datos.

```python
import numpy as np

housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)
```

```python
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
```

**Diagrama de flujo:**

```mermaid
flowchart TD
    A[Datos originales] --> B[Crear columna income_cat]
    B --> C[StratifiedShuffleSplit]
    C --> D[Entrenamiento]
    C --> E[Test]
```

---

## 5. Validar la estratificación

Comprobamos que las proporciones de las categorías en el conjunto de testeo son similares a las del conjunto original.

```python
strat_test_set["income_cat"].value_counts() / len(strat_test_set)
```

---

## 6. Eliminar la columna auxiliar

Una vez hecha la separación, eliminamos la columna `income_cat` para no usarla en el modelo.

```python
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
```

---

## 7. Visualización y exploración

Podemos visualizar los datos para entender mejor su distribución geográfica, por ejemplo:

```python
housing = strat_train_set.copy()
housing.plot(kind="scatter", x="longitude", y="latitude")
```

---

## Resumen: Receta de implementación

1. **Cargar los datos**.
2. **Crear columna categórica** si es necesario para estratificación.
3. **Separar los datos** usando aleatorización, hash o estratificación.
4. **Validar proporciones** en los conjuntos.
5. **Eliminar columnas auxiliares**.
6. **Explorar y visualizar** los datos de entrenamiento.
7. **Entrenar y evaluar** el modelo usando los conjuntos separados.

---

## Diagrama general del proceso

```mermaid
graph TD
    A[Datos originales]
    A --> B[Crear columna categórica]
    B --> C[Separar datos (aleatorio/hash/estratificado)]
    C --> D[Validar proporciones]
    D --> E[Eliminar columnas auxiliares]
    E --> F[Explorar datos]
    F --> G[Entrenar modelo]
    F --> H[Testear modelo]
```

---

**¡Listo! Así puedes implementar y validar la separación de tus datos para Machine Learning de forma profesional y reproducible.**